In [1]:
%cd /Users/jonathancheng/PycharmProjects/espnff/src/


/Users/jonathancheng/PycharmProjects/espnff/src


In [2]:
import nfl_schedule as nf
import ff_league_data as ff

In [3]:
year_of_interest=2022

df=nf.get_nfl_schedule(year_of_interest)

In [4]:
import os
import requests
from bs4 import BeautifulSoup

import json
import numpy as npa
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.io as pio

# import datetime
from datetime import date
from datetime import datetime

import requests
import espn_api
from espn_api.football import League
from espn_api.football import activity

import altair as alt

# import sportsipy.nfl
# from sportsipy.nfl.boxscore import Boxscore
# from sportsipy.nfl.schedule import Schedule
# from sportsipy.nfl.teams import Teams
import nfl_data_py as nfl

# from sportsipy.nfl.teams import Teams

# teams = Teams()
# lions = teams('DET')


In [5]:
path = r'/Users/jonathancheng/PycharmProjects/espnff/data'

In [6]:
def mine_nfl_schedule(year_of_interest: int): 
    """_summary_

    Args:
        year_of_interest (_type_): _description_

    Returns:
        _type_: _description_
    """
    url = f'https://www.pro-football-reference.com/years/{year_of_interest}/games.htm'
    return pd.read_html(url)[0]


In [ ]:
df_proteam_names = pd.read_csv(os.path.join(path,'team_abbrev_conversion.csv'))


In [ ]:
df = mine_nfl_schedule(year_of_interest=2022)

df = df[['Week','Date','Winner/tie','Loser/tie']]
df = pd.melt(df, id_vars=['Week','Date'],value_vars=['Winner/tie','Loser/tie'],var_name='Winner_Loser',value_name='Pro_team')

df = df.dropna(subset='Pro_team',axis=0)
# df['Pro_team'] = df['Pro_team'].str.replace(pat=r'^Washington.*', repl='Washington')
df['Pro_team'] = df['Pro_team'].str.replace(pat=r'^Washington.*', repl='Washington', regex=True)

In [ ]:
df['Pro_team'].unique()

In [ ]:
import difflib as dfl

In [ ]:
dfl.get_close_matches()

In [ ]:
df2['team'].apply(lambda x: difflib.get_close_matches(x, df1['team'])[0])

In [ ]:
df[df['Pro_team'].str.contains('Washington')]

In [ ]:

df['Pro_team_abbrev'] = df['Pro_team'].map(dict(zip(df_proteam_names['Pro Team Name'],df_proteam_names['Abbrev'])))
df = df[df['Pro_team'].isin(df_proteam_names['Pro Team Name'].tolist())]


In [ ]:
def format_nfl_schedule(df, df_proteam_names):
    """_summary_

    Args:
        df (_type_): _description_

    Returns:
        _type_: _description_
    """

    df = df[['Week','Date','Winner/tie','Loser/tie']]
    df = pd.melt(df, id_vars=['Week','Date'],value_vars=['Winner/tie','Loser/tie'],var_name='Winner_Loser',value_name='Pro_team')
    df['Pro_team_abbrev'] = df['Pro_team'].map(dict(zip(df_proteam_names['Pro Team Name'],df_proteam_names['Abbrev'])))
    df = df[df['Pro_team'].isin(df_proteam_names['Pro Team Name'].tolist())]

    # weeks = [int(i) for i in df['Week'].unique().tolist() if i.isdigit()]
    # games_all_season = max(weeks)

    return df

def get_wideform_nfl_schedule(df):

    """_summary_

    Returns:
        _type_: _description_
    """

    weeks = [int(i) for i in df['Week'].unique().tolist() if i.isdigit()]
    games_all_season = max(weeks)

    df = df[df['Week'].isin([str(i) for i in weeks])]
    df = df.astype({'Week':int})
    # pivot NFL schedule to wideform
    df = df[['Week','Date','Pro_team_abbrev']]
    df = df.pivot(index='Week',columns='Pro_team_abbrev',values='Date').fillna('BYE')
    df = df.applymap(lambda x: pd.to_datetime(x,format='%Y-%m-%d') if x!='BYE' else x)
    return df
    
def get_nfl_schedule(year_of_interest, 
                    data_path=r'/Users/jonathancheng/PycharmProjects/espnff/data'):
    """_summary_

    Args:
        year_of_interest (_type_): _description_
        data_path (regexp, optional): _description_. Defaults to r'/Users/jonathancheng/PycharmProjects/espnff/data'.

    Returns:
        _type_: _description_
    """

    df_proteam_names = pd.read_csv(os.path.join(data_path,'team_abbrev_conversion.csv'))

    return (mine_nfl_schedule(year_of_interest=year_of_interest)
            .pipe(format_nfl_schedule, df_proteam_names)
            .pipe(get_wideform_nfl_schedule))


In [ ]:
df_proteam_names = pd.read_csv(os.path.join(path,'team_abbrev_conversion.csv'))

(mine_nfl_schedule(year_of_interest=year_of_interest)
    .pipe(format_nfl_schedule, df_proteam_names)
    .pipe(get_wideform_nfl_schedule))

In [ ]:
# Football Import

league_id = 1094090
# year = 2022

SWID="{F191FB8C-DB2D-4D24-91FB-8CDB2DED249D}"

s2='AECJMQHsUHB0FTXdZkw93uY7GRbX8BPnm93Ye6AwvwrMsrZFGg1Lbmi07SWVov2ioN8zGMFDzZiiDSeQCa7WQHaGivGnMfGWLjmfGwkOeLXb5baD1sltp%2B%2BIfHAtl98TpmHgB16ZpGn6g3Bm5vLEA7yDC6HkbD3LSp0E2rGB7hKziLMvZ7mT6ONJFRe8Xp3ApYWSvxPr9cz0pJiI%2FF0blsZ8hyATDJMEyaQ2O%2FypcsViORr6hqYTmXHPuPKnMBfvYC8LQqi1exGw3vnyg6ptsB2Y'

In [ ]:
league = League(league_id=league_id, year=year_of_interest, espn_s2=s2, swid=SWID)

activity_ls = league.recent_activity(1000)

In [ ]:
def fetch_espn_api(league_id, year, espn_s2, swid):
    """Uses espn_api to fetch league data

    Args:
        league_id (_type_): ESPN League ID
        year (_type_): Year of season of interest
        espn_s2 (_type_): _description_
        swid (_type_): _description_

    Returns:
        list: recent activity list generated by espn_api
    """
    return League(league_id=league_id, year=year_of_interest, espn_s2=s2, swid=SWID).recent_activity(100000000)


In [ ]:
activity_ls = fetch_espn_api(league_id=league_id, year=year_of_interest, espn_s2=s2, swid=SWID)

In [ ]:
activity_ls[0]

In [ ]:
for k in activity.actions:
    (team1, action1, _, _) = k
    team_id = team1.team_name
    team_transaction_ls.append(team_id)

[(team1.team_name, action1, _, _) for ]

In [ ]:
acq_data_ls = []

# loop through each activity item
for idx, j in enumerate(activity_ls):
    date1 = j.date
    team_transaction_ls = []
    for k in j.actions:
        (team1, action1, _ , _ ) = k
        team_id = team1.team_name
        team_transaction_ls.append(team_id)

In [ ]:
k

In [ ]:
[(team1, action1, player_name, bid_amount) for k in j.actions]

In [ ]:
k

In [ ]:
import datetime

# Function handling trade data

def get_trade_data(j):
    "Function for parsing Recent Activities trade data into ADD and DROP events"
    
    acq_data_ls = []

    # date1 = j.date

    team_transaction_ls = []
    
    for k in j.actions:
        (team1, action1, _ , _ ) = k
        team_id = team1.team_name
        team_transaction_ls.append(team_id)

        # determine if trade or waiver acquisition type

    # establish other team involved in trade
    teams_in_trade = list(set(team_transaction_ls))

    for k in j.actions:
        (team1, action1, player_name, bid_amount) = k
        team_id = team1.team_name

        if teams_in_trade.index(team_id) == 1:
            team_id_received = teams_in_trade[0]
        else:
            team_id_received = teams_in_trade[1]

        # Build new tuple for TRADE ADDED
        player_id = player_name.name
        ProTeam = player_name.proTeam
        action1 = 'TRADE ADDED'

        # Build a second tuple for TRADE DROPPED
        action2 = 'TRADE DROPPED'

        acq_data1 = (date1 , player_id , ProTeam , team_id_received , action1, bid_amount)
        drop_data1 = (date1 , player_id , ProTeam , team_id , action2, bid_amount)

        acq_data_ls.append(acq_data1)
        acq_data_ls.append(drop_data1)
        
    return acq_data_ls
    
# Function handling waiver data
def get_waiver_data(j):
    
    waiver_data_ls = []

    for k in j.actions:
        (team1, action1, player_name, bid_amount) = k
        team_id = team1.team_name
        player_id = player_name.name
        ProTeam = player_name.proTeam
        
        waiver_data1 = (date1 , player_id , ProTeam , team_id , action1 , bid_amount)
        waiver_data_ls.append(waiver_data1)
        
    return waiver_data_ls

In [ ]:
def flatten_list(_2d_list):
    flat_list = []
    # Iterate through the outer list
    for element in _2d_list:
        if type(element) is list:
            # If the element is of type list, iterate through the sublist
            for item in element:
                flat_list.append(item)
        else:
            flat_list.append(element)
    return flat_list

In [ ]:
# Loop thru Recent Activities data, creating a DataFrame for all acquisitions
# Conditional for trade or waiver acquisitions

acq_data_ls = []

# loop through each activity item
for idx, j in enumerate(activity_ls):
    date1 = j.date
    team_transaction_ls = []
    for k in j.actions:
        (team1, action1, _ , _ ) = k
        team_id = team1.team_name
        team_transaction_ls.append(team_id)
        
        # determine if trade or waiver acquisition type        
        ls = []
        
        if action1 == 'TRADED':
            ls = get_trade_data(j)

        else:
            ls = get_waiver_data(j)

    acq_data_ls.append(ls)
      

acq_data_flat_ls = flatten_list(acq_data_ls)

df_acq = pd.DataFrame(acq_data_flat_ls)

df_acq.columns = ['Timestamp','Player','ProTeam','Team','Action','Bid Amount ($)']

# converts from milliseconds to date
def convert_to_date(date1):
    d2=datetime.date.fromtimestamp(date1 / 1000.0)
    return d2

df_acq['Action Timestamp'] = df_acq['Timestamp'].apply(lambda x : convert_to_date(x))

df_acq = df_acq.sort_values(by=['Timestamp'],ascending=True)

In [ ]:
import datetime

# Function handling trade data

def get_trade_data(j,date1):
    "Function for parsing Recent Activities trade data into ADD and DROP events"
    
    acq_data_ls = []

    # date1 = j.date

    team_transaction_ls = []
    
    for k in j.actions:
        (team1, action1, _ , _ ) = k
        team_id = team1.team_name
        team_transaction_ls.append(team_id)

        # determine if trade or waiver acquisition type

    # establish other team involved in trade
    teams_in_trade = list(set(team_transaction_ls))

    for k in j.actions:
        (team1, action1, player_name, bid_amount) = k
        team_id = team1.team_name

        if teams_in_trade.index(team_id) == 1:
            team_id_received = teams_in_trade[0]
        else:
            team_id_received = teams_in_trade[1]

        # Build new tuple for TRADE ADDED
        player_id = player_name.name
        ProTeam = player_name.proTeam
        action1 = 'TRADE ADDED'

        # Build a second tuple for TRADE DROPPED
        action2 = 'TRADE DROPPED'

        acq_data1 = (date1 , player_id , ProTeam , team_id_received , action1, bid_amount)
        drop_data1 = (date1 , player_id , ProTeam , team_id , action2, bid_amount)

        acq_data_ls.append(acq_data1)
        acq_data_ls.append(drop_data1)
        
    return acq_data_ls
    
# Function handling waiver data
def get_waiver_data(j,date1):
    
    waiver_data_ls = []

    for k in j.actions:
        (team1, action1, player_name, bid_amount) = k
        team_id = team1.team_name
        player_id = player_name.name
        ProTeam = player_name.proTeam
        
        waiver_data1 = (date1 , player_id , ProTeam , team_id , action1 , bid_amount)
        waiver_data_ls.append(waiver_data1)
        
    return waiver_data_ls
def get_acq_ls(activity_ls):

    acq_data_ls = []

    # loop through each activity item
    for idx, j in enumerate(activity_ls):
        date1 = j.date
        team_transaction_ls = []
        for k in j.actions:
            (team1, action1, _ , _ ) = k
            team_id = team1.team_name
            team_transaction_ls.append(team_id)
            
            # determine if trade or waiver acquisition type        
            ls = []
            
            if action1 == 'TRADED':
                ls = get_trade_data(j,date1)

            else:
                ls = get_waiver_data(j,date1)

        acq_data_ls.append(ls)

    return flatten_list(acq_data_ls)


In [ ]:
ls2 = get_acq_ls(activity_ls)

In [ ]:
acq_data_flat_ls==ls2

In [ ]:
df_acq

In [ ]:
def get_waiver_data(activity):

    date1 = activity.date
    
    waiver_data_ls = []

    for k in activity.actions:
        (team1, action1, player_name, bid_amount) = k
        team_id = team1.team_name
        player_id = player_name.name
        ProTeam = player_name.proTeam
        
        waiver_data1 = (date1 , player_id , ProTeam , team_id , action1 , bid_amount)
        waiver_data_ls.append(waiver_data1)
        
    return waiver_data_ls

In [ ]:
# Loop thru Recent Activities data, creating a DataFrame for all acquisitions
# Conditional for trade or waiver acquisitions

acq_data_ls = []

# loop through each activity item
for idx, j in enumerate(activity_ls):
    date1 = j.date
    team_transaction_ls = []
    for k in j.actions:
        (team1, action1, _ , _ ) = k
        team_id = team1.team_name
        team_transaction_ls.append(team_id)
        
        # determine if trade or waiver acquisition type        
        ls = []
        
        if action1 == 'TRADED':
            ls = get_trade_data(j)

        else:
            ls = get_waiver_data(j)

    acq_data_ls.append(ls)
      

acq_data_flat_ls = flatten_list(acq_data_ls)

df_acq = pd.DataFrame(acq_data_flat_ls)

df_acq.columns = ['Timestamp','Player','ProTeam','Team','Action','Bid Amount ($)']

# converts from milliseconds to date
def convert_to_date(date1):
    d2=datetime.date.fromtimestamp(date1 / 1000.0)
    return d2

df_acq['Action Timestamp'] = df_acq['Timestamp'].apply(lambda x : convert_to_date(x))

df_acq = df_acq.sort_values(by=['Timestamp'],ascending=True)

In [ ]:
df_acq.sort_values(by=['Bid Amount ($)'],ascending=False).iloc[0:20]

## Get Draft Data

In [ ]:
draft_ls = []

for i in league.draft:    
    team_id = i.team.team_name
    player_id = i.playerName
    bid_amount = i.bid_amount
    action = 'DRAFT ADDED'
    draft_ls.append((player_id, team_id,action,bid_amount))

df_draft = pd.DataFrame(draft_ls)
  
df_draft.columns = ['Player', 'Acquired by' , 'Action' , 'Bid Amount ($)']

df_draft.set_index('Player',inplace=True)

In [ ]:
df_draft.reset_index()

## Get total points of rostered players Dataframe

In [ ]:
# Get data of all currently fantasy league rostered players

rostered_players_ls = []
for i in league.teams:
    team_name = i.team_name
    for player in i.roster:
        player_name = player.name
        player_pos = player.position
        player_pro_team = player.proTeam
        player_total_pts = player.total_points
        player_tup = (player_name , player_pos, player_pro_team, player_total_pts , team_name)
        rostered_players_ls.append(player_tup)

df_rostered = pd.DataFrame(rostered_players_ls)
col_ls = ['Player' , 'Position' , 'Pro Team', 'Total points', 'Current Team']
df_rostered.columns = col_ls

df_rostered.set_index('Player',inplace=True)

In [ ]:
df_rostered.reset_index().sort_values(by='Total points',ascending=False)

In [ ]:
from datetime import datetime

free_agent_ls = league.free_agents(size=1000)

FA_players_ls = []
name_ls = []
for i in free_agent_ls:
    team_name = 'Free agent'
    player_name = i.name
    player_pos = i.position
    player_pro_team = i.proTeam
    player_total_pts = i.total_points

    player_tup = (player_name , player_pos, player_pro_team, player_total_pts , team_name)
    FA_players_ls.append(player_tup)
    
    name_ls.append(player_name)


df_FA = pd.DataFrame(FA_players_ls)
col_ls = ['Player' , 'Position' , 'Pro Team', 'Total points', 'Current Team']
df_FA.columns=col_ls
df_FA.set_index('Player',inplace=True)

# Generate all player stats dataframe, including all Free Agents
df_player_stats = pd.concat([df_rostered,df_FA],axis=0)

df_draft_acq = df_draft.join(df_player_stats)

df_draft_acq['Player'] = df_draft_acq.index

col_new = ['Player' , 'Pro Team' , 'Acquired by' , 'Action' , 'Bid Amount ($)']

df_draft_acq = df_draft_acq[col_new]

df_draft_acq.rename(columns = {'Pro Team' : 'ProTeam' , 'Acquired by' : 'Team'} , inplace = True)


# Establish Draft Date
# Draft Date just needs to be before the first game of the season, 
# By default, we set this arbitrarily as one day before the first game of the NFL season

draft_date1 = season_start_date - timedelta(days=1)

draft_date_str = draft_date1.strftime('%d %b %Y')

draft_date = datetime.strptime(draft_date_str, '%d %b %Y').date()

draft_date_ms = int(datetime.strptime(draft_date_str, '%d %b %Y').timestamp() * 1000)

df_draft_acq['Action Timestamp'] = draft_date

df_draft_acq['Timestamp'] = draft_date_ms

df_acq_final = pd.concat([df_draft_acq , df_acq] , axis=0)

df_acq_final.reset_index(inplace=True)

df_acq_final.drop('index' , inplace=True ,axis=1)